In [585]:
import pygame, sys, math, pygame.gfxdraw
pygame.init()
image = pygame.image.load("Wall2.jpg")

In [586]:
class Cam:
    def __init__(self, image, pos = (5, 14, 0), rot = (0, 90)):
        self.pos = list(pos)
        self.rot = list(rot)
        self.v = 1/10
        self.v2 = 0.5/4
        self.p = 1/3
        self.yc = 450/2
        self.y = 450/2
        self.floor = self.y
        self.l = 3
        self.vy = 0
        self.image = image
    def update(self, key, dt):
        if key[pygame.K_w]: self.pos[1] += self.v * math.sin(math.radians(self.rot[0] + self.rot[1] / 2)); self.pos[0] += self.v * math.cos(math.radians(self.rot[0] + self.rot[1] / 2)) 
        if key[pygame.K_s]: self.pos[1] -= self.v * math.sin(math.radians(self.rot[0] + self.rot[1] / 2)); self.pos[0] -= self.v * math.cos(math.radians(self.rot[0] + self.rot[1] / 2)) 
        if key[pygame.K_a]: self.pos[0] += self.v * math.cos(math.radians(90 - self.rot[0] - self.rot[1] / 2)); self.pos[1] -= self.v * math.sin(math.radians(90 - self.rot[0] - self.rot[1] / 2))
        if key[pygame.K_d]: self.pos[0] -= self.v * math.cos(math.radians(90 - self.rot[0] - self.rot[1] / 2)); self.pos[1] += self.v * math.sin(math.radians(90 - self.rot[0] - self.rot[1] / 2))
        if key[pygame.K_q]: self.rot[0] -= 2
        if key[pygame.K_e]: self.rot[0] += 2
        if key[pygame.K_LSHIFT]: self.v = 1/3
        if not key[pygame.K_LSHIFT]: self.v = 1
        #if not key[pygame.K_LCTRL]: self.y = self.yc - 50; self.l = 15
        if key[pygame.K_SPACE]: self.vy = 6
        self.y += self.vy
        #self.vy += (-15.8)*dt
        self.rot[0] %= 360
        self.rot[1] %= 360
    def events(self, event):
        if event.type == pygame.MOUSEMOTION:
            x, y = event.rel
            x //= 15
            self.rot[0] += x
            self.y -= y
    def box(self, k):
        if self.pos[0] <= 0: self.pos[0] = 1
        if self.pos[1] <= 0: self.pos[1] = 1
        if self.pos[0] >= k: self.pos[0] = k-1
        if self.pos[1] >= k: self.pos[1] = k-1
        #if self.y <= self.yc: self.y = self.yc; self.vy =  0
    def check(self, struct, alp, m, dis, v, p, floor, k, i, x, l):
            x1 = self.pos[0] + math.cos(math.radians(alp))*dis
            y1 = self.pos[1] + math.sin(math.radians(alp))*dis
            Ax = struct.pos[i][x%len(struct.pos[i])][0]
            Ay = struct.pos[i][x%len(struct.pos[i])][1]
            Bx = struct.pos[i][(x+1)%len(struct.pos[i])][0]
            By = struct.pos[i][(x+1)%len(struct.pos[i])][1]
            Cx = struct.pos[i][(x+2)%len(struct.pos[i])][0]
            Cy = struct.pos[i][(x+2)%len(struct.pos[i])][1]
            t1 = (Ax - x1)*(By - Ay) - (Bx - Ax)*(Ay - y1)
            t2 = (Bx - x1)*(Cy - By) - (Cx - Bx)*(By - y1)
            t3 = (Cx - x1)*(Ay - Cy) - (Ax - Cx)*(Cy - y1)
            if t1>0 and t2>0 and t3>0: 
                    if k == 0: v = v/2; m += 1
                    dis -= v
                    k = 1
                    if m < 4:
                        (dis) = self.check(struct, alp, m, dis, v, p, floor, k, i, x, l)
                    #else: pygame.draw.rect(screen, (170 - 170*8/10*dis/424, 30 - 30*8/10*dis/424, 100 - 100*8/10*dis/424), ((alp - self.rot[0])*800/self.rot[1], floor - l * 800/dis, (p) * 800/self.rot[1] + 1, l*2*800/dis))
                        
            elif t1<0 and t2<0 and t3<0:
                    if k == 0: v = v/2; m += 1
                    dis -= v
                    k = 1
                    if m < 4:
                        (dis) = self.check(struct, alp, m, dis, v, p, floor, k, i, x, l)
                    #else: 
                        

            else:
                if m > 0 and k == 1: 
                    v = v/2
                dis += v
                k = 0
                if dis < 141: (dis) = self.check(struct, alp, m, dis, v, p, floor, k, i, x, l)
            return (dis)
    def Map(self, k, alp, floor, p, image, size):
        Alp = alp%360 
        dis1 = 0
        rect = (image.get_rect().width, image.get_rect().height) 
        alp1 = (-math.degrees(math.atan(self.pos[0]/self.pos[1])) - 90)%360
        alp2 = (-math.degrees(math.atan(self.pos[1]/(k - self.pos[0]))))%360
        alp3 = (math.degrees(math.atan((k - self.pos[1])/(k - self.pos[0]))))%360
        alp4 = (math.degrees(math.atan(self.pos[0]/(k - self.pos[1])))+90)%360
        q = 10
        if Alp >= alp1 and Alp <= alp2:
            dis = math.sqrt(self.pos[1] ** 2 + ((1/math.tan(math.radians(Alp)))*self.pos[1]) ** 2)
            dis = dis*math.cos(math.radians(Alp - self.rot[0] - self.rot[1] / 2))
            if Alp >= 270: b = self.pos[0] + abs(1/math.tan(math.radians(Alp)))*self.pos[1]
            else: b = self.pos[0] - abs(1/math.tan(math.radians(Alp)))*self.pos[1]
            if ((self.p) * size[0]/self.rot[1] + (q*(b)*rect[0]/k)%rect[0]) <= rect[0]:
                crop = pygame.Surface(((self.p) * size[0]/self.rot[1] + 1, rect[1]))
                crop.fill((155, 50, 155))   
                crop.blit(image, (0, 0), ((q*(b)*rect[0]/k)%rect[0], 0, (self.p) * size[0]/self.rot[1] + 1, rect[1]))
            else:
                crop = pygame.Surface((rect[0] - (q*(b)*rect[0]/k)%rect[0], rect[1]))
                crop.fill((155, 50, 155))
                crop.blit(image, (0, 0), ((q*(b)*rect[0]/k)%rect[0], 0, rect[0] - (q*(b)*rect[0]/k)%rect[0], rect[1]))
                crop = pygame.transform.scale(crop, (int((self.p) * size[0]/self.rot[1]), rect[1]))
            imager = pygame.transform.scale(crop, (int((self.p) * size[0]/self.rot[1] + 1), int(self.l*2*(size[1] - self.l)/dis)))
            screen.blit(imager, ((Alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis))
            imager.fill((24, 27, 15))
            imager.set_alpha(3*dis)
            screen.blit(imager, ((Alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis))
            del(crop, imager)
        if Alp >= alp2 or Alp <= alp3:
            dis = math.sqrt(((math.tan(math.radians(Alp)))*(k - self.pos[0])) ** 2 + (k - self.pos[0]) ** 2)
            #if Alp >= 180: b = self.pos[1] - abs(math.tan(math.radians(Alp)))*self.pos[0]
            dis = dis*math.cos(math.radians(Alp%360 - self.rot[0]%360 - self.rot[1] / 2))
            #pygame.gfxdraw.box(screen, ((Alp%360 - self.rot[0]%360)*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis, (self.p) * size[0]/self.rot[1] + 1, self.l*2*(size[1] - self.l)/dis), (200 - (200*9/10)*(dis/(k*math.sqrt(2))), 70 - (70*9/10)*dis/(k*math.sqrt(2)), 150 - (150*9/10)*dis/(k*math.sqrt(2))))
            pygame.draw.rect(screen, (200 - (200*5/10)*(dis/(k*math.sqrt(2))), 70 - (70*5/10)*dis/(k*math.sqrt(2)), 150 - (150*5/10)*dis/(k*math.sqrt(2))), ((alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis, (self.p) * size[0]/self.rot[1] + 1, self.l*2*(size[1] - self.l)/dis))
            #screen.blit(image, ((Alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis),  ((Alp - alp2)*rect[0]/(alp3-alp2), 0, ((Alp - alp2)*rect[0]/(alp3-alp2) + (alp3-alp2)*self.p, rect[1])))
        if Alp >= alp3 and Alp <= alp4:
            dis = math.sqrt((k - self.pos[1]) ** 2 + (1/math.tan(math.radians(Alp))*(k - self.pos[1])) ** 2)
            dis = dis*math.cos(math.radians(Alp%360 - self.rot[0]%360 - self.rot[1] / 2))
            #pygame.gfxdraw.box(screen, ((Alp%360 - self.rot[0]%360)*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis, (self.p) * size[0]/self.rot[1] + 1, self.l*2*(size[1] - self.l)/dis), (200 - (200*9/10)*(dis/(k*math.sqrt(2))), 70 - (70*9/10)*dis/(k*math.sqrt(2)), 150 - (150*9/10)*dis/(k*math.sqrt(2))))
            pygame.draw.rect(screen, (200 - (200*5/10)*(dis/(k*math.sqrt(2))), 70 - (70*5/10)*dis/(k*math.sqrt(2)), 150 - (150*5/10)*dis/(k*math.sqrt(2))), ((alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis, (self.p) * size[0]/self.rot[1] + 1, self.l*2*(size[1] - self.l)/dis))
            #screen.blit(image, ((alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis),  ((Alp - alp3)*rect[0]/(alp4-alp3), 0, ((Alp - alp3)*rect[0]/(alp4-alp3) + (alp4-alp3)*self.p, rect[1])))
        if Alp >= alp4 and Alp <= alp1:
            dis = math.sqrt(((math.tan(math.radians(Alp)))*self.pos[0]) ** 2 + (self.pos[0]) ** 2)
            dis = dis*math.cos(math.radians(Alp%360 - self.rot[0]%360 - self.rot[1] / 2))
            #pygame.gfxdraw.box(screen, ((Alp%360 - self.rot[0]%360)*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis, (self.p) * size[0]/self.rot[1] + 1, self.l*2*(size[1] - self.l)/dis), (200 - (200*9/10)*(dis/(k*math.sqrt(2))), 70 - (70*9/10)*dis/(k*math.sqrt(2)), 150 - (150*9/10)*dis/(k*math.sqrt(2))))
            pygame.draw.rect(screen, (200 - (200*5/10)*(dis/(k*math.sqrt(2))), 70 - (70*5/10)*dis/(k*math.sqrt(2)), 150 - (150*5/10)*dis/(k*math.sqrt(2))), ((alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis, (self.p) * size[0]/self.rot[1] + 1, self.l*2*(size[1] - self.l)/dis))
            #screen.blit(image, ((alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis),  ((Alp - alp4)*rect[0]/(alp1-alp4), 0, ((Alp - alp4)*rect[0]/(alp1-alp4) + (alp1-alp4)*self.p, rect[1])))
        
       # pygame.gfxdraw.box(screen, ((alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis, (self.p) * size[0]/self.rot[1] + 1, self.l*2*(size[1] - self.l)/dis), (200 - (200*9/10)*(dis/(k*math.sqrt(2))), 70 - (70*9/10)*dis/(k*math.sqrt(2)), 150 - (150*9/10)*dis/(k*math.sqrt(2))))
        #pygame.draw.rect(screen, (200 - (200*5/10)*(dis/(k*math.sqrt(2))), 70 - (70*5/10)*dis/(k*math.sqrt(2)), 150 - (150*5/10)*dis/(k*math.sqrt(2))), ((alp - self.rot[0])*size[0]/self.rot[1], floor - self.l*(size[1] - self.l)/dis, (self.p) * size[0]/self.rot[1] + 1, self.l*2*(size[1] - self.l)/dis))
        if Alp <= alp1 + 0.25 and Alp >= alp1 - 0.25:
            dis1 = math.sqrt((self.pos[1])**2 + (1/math.tan(math.radians(alp1))*self.pos[1])**2)
            pygame.draw.line(screen, (0, 0, 0), ((alp1-self.rot[0])*size[0]/self.rot[1], floor - self.l*size[1]/dis1), ((alp1-self.rot[0])*size[0]/self.rot[1], floor + self.l*size[1]/dis1))
        if Alp <= alp2 + 0.25 and Alp >= alp2 - 0.25:
            dis1 = math.sqrt((self.pos[1])**2 + (1/math.tan(math.radians(alp2))*self.pos[1])**2)
            pygame.draw.line(screen, (0, 0, 0), ((alp2-self.rot[0])*size[0]/self.rot[1], floor - self.l*size[1]/dis1), ((alp2-self.rot[0])*size[0]/self.rot[1], floor + self.l*size[1]/dis1))
        if Alp <= alp3 + 0.25 and Alp >= alp3 - 0.25:
            dis1 = math.sqrt((k - self.pos[0])**2 + (k - self.pos[1])**2)
            pygame.draw.line(screen, (0, 0, 0), (((alp3-self.rot[0]%360)%360)*size[0]/self.rot[1], floor - self.l*size[1]/dis1), (((alp3-self.rot[0]%360)%360)*size[0]/self.rot[1], floor + self.l*size[1]/dis1))
        if Alp <= alp4 + 0.25 and Alp >= alp4 - 0.25:
            dis1 = math.sqrt((self.pos[0])**2 + (math.tan(math.radians(alp4))*(self.pos[0]))**2)
            pygame.draw.line(screen, (0, 0, 0), ((alp4-self.rot[0])*size[0]/self.rot[1], floor - self.l*size[1]/dis1), ((alp4-self.rot[0])*size[0]/self.rot[1], floor + self.l*size[1]/dis1))
        #if dis1 != 0: pygame.draw.line(screen, (0, 0, 0), ((Alp-self.rot[0])*size[0]/self.rot[1], floor - self.l*size[1]/dis1), ((Alp%360-self.rot[0]%360)*size[0]/self.rot[1], floor + self.l*size[1]/dis1))
    def draw_self(self, struct, size):
        k = 100
        screen.fill(0)
        pygame.draw.rect(screen, (130, 130, 255), (0, self.y, size[0], -self.y))
        alp = self.rot[0]%360
        while alp <= self.rot[0] + self.rot[1]:
            self.Map(k, alp, self.y, self.p, self.image, size)
            dislist = []
            for i in range(len(struct.pos)):
                for x in range(0, len(struct.pos[i]), 1):
                    dis = self.check(struct, alp*self.p, 0, 0, 5, self.p, self.y, 0, i, x, self.l)
                    if dis <= 113: dislist.append(dis); del(dis)
            dislist.sort(reverse=True)
            q = len(dislist)-1
            if len(dislist) > 0: #pygame.gfxdraw.box(screen, ((alp*self.p - self.rot[0])*800/self.rot[1], self.y - self.l * 800/dislist[q], (self.p) * 800/self.rot[1] + 1, self.l*2*800/dislist[q]), (170 - 170*8/10*dislist[q]/(700*math.sqrt(2)), 30 - 30*8/10*dislist[q]/(700*math.sqrt(2)), 100 - 100*8/10*dislist[q]/(700*math.sqrt(2))))
                pygame.draw.rect(screen, (170 - 170*8/10*dislist[q]/(700*math.sqrt(2)), 30 - 30*8/10*dislist[q]/(700*math.sqrt(2)), 100 - 100*8/10*dislist[q]/(700*math.sqrt(2))), ((alp*self.p%360 - self.rot[0]%360)*800/self.rot[1], self.y - self.l * 800/dislist[q], (self.p) * 800/self.rot[1] + 1, self.l*2*800/dislist[q]))
            alp += self.p
    def draw_(self):
        k = 100
        pygame.draw.rect(screen, (170, 30, 100), (0,  0, 300/3, 300/3), 1)
        #pygame.draw.rect(screen, (0, 0, 0), (0, 0, 300/3, 300/3))
        pygame.draw.line(screen, (35, 135, 255), (int(self.pos[0]), int(self.pos[1])), ((self.pos[0] + math.cos(math.radians(self.rot[0]%360))*25), (self.pos[1] + math.sin(math.radians(self.rot[0]))*25)), 1)
        pygame.draw.line(screen, (35, 135, 255), (int(self.pos[0]), int(self.pos[1])), ((self.pos[0] + math.cos(math.radians(self.rot[0] + self.rot[1]))*25), (self.pos[1] + math.sin(math.radians(self.rot[0] + self.rot[1]))*25)), 1)
        pygame.draw.circle(screen, (235, 0, 55), (int(self.pos[0]), int(self.pos[1])), 4)

In [587]:
class Structure:
    def __init__(self):
        self.pos =[]# [(50, 50), (50, 100), (100, 100), (100, 50)]]#, 
                   # [(140, 200), (240, 200), (240, 300), (140, 300)]] 
                    #[(490, 10), (490, 520), (520, 520), (520, -20)], 
                    #[(-20, 10), (10, 10), (10, 520), (-20, 520)],
                    #[(-20, 490), (490, 490), (490, 520), (-20, 520)]]
        #self.pos = [(100, 100), (300, 100), (100, 500)]
        self.fakepos = []
        for i in range(len(self.pos)):
            self.fakepos.append([])
            for x in range(len(self.pos[i])):
                self.fakepos[i].append((self.pos[i][x][0] / 3, self.pos[i][x][1] / 3))
        for i in range(len(self.pos)):
            for x in range(len(self.pos[i])):
                self.pos[i][x] = (self.pos[i][x][0], self.pos[i][x][1])
        self.per = []
    def draw(self):
        for i in range(len(self.pos)):
            pygame.draw.polygon(screen, (170, 30, 100), self.fakepos[i])
    def summ(self):
        for i in range(1, len(self.pos)):
            for x in range(len(self.pos[i])):
                self.per.append(math.sqrt((self.pos[i][x][0] - self.pos[x][x - 1][0])**2 + (self.pos[i][x][1] - self.pos[i][x-1][1])**2))
            self.per.append(math.sqrt((self.pos[i][len(self.pos) - 1][0] - self.pos[i][0][0])**2 + (self.pos[i][len(self.pos) - 1][1] - self.pos[0][1])**2))
            self.s = math.sqrt(sum(self.per) / 2 * (sum(self.per) / 2 - self.per[0])* (sum(self.per) / 2 - self.per[1])* (sum(self.per) / 2 - self.per[2]))

In [588]:
print(math.degrees(math.radians(535.6786875%360)))

175.67868750000002


In [589]:
k = True
size = (1500, 800)
q = (40, 40)

In [590]:
clock = pygame.time.Clock()
cam = Cam(image) 
screen = pygame.display.set_mode(size) 
struct = Structure()
#struct.summ()
pygame.event.get(); pygame.mouse.get_rel()
pygame.mouse.set_visible(0); pygame.event.set_grab(1)
dt = clock.tick()/1000
while k: 
    clock.tick(20) 
    for event in pygame.event.get(): 
        if event.type == pygame.QUIT: 
            k = False 
        elif event.type == pygame.KEYDOWN: 
            if event.key == pygame.K_ESCAPE: k = False
        cam.events(event) 
    key = pygame.key.get_pressed()
    cam.box(300/3)
    struct.draw()
    cam.draw_self(struct, size)
    cam.update(key, dt)
    cam.draw_()
    pygame.display.flip()
    screen.fill((255, 255, 255))
pygame.quit()
sys.exit()

SystemExit: 

In [ ]:
k = True
size = (1500, 800)
q = (40, 40)

In [ ]:
import pygame
pygame.init()
screen = pygame.display.set_mode(size) 
image = pygame.image.load("back1.png")
#dt = clock.tick()/1000
crop = pygame.Surface((180, 180))
crop.blit(image, (0, 0), (0, 0, 180, 180))
crop.set_colorkey((0, 0, 0))
crop.set_alpha(50)
imager = pygame.transform.scale(crop, (10, 100))
pygame.display.update()
k = True
while k: 
    clock.tick(20) 
    for event in pygame.event.get(): 
        if event.type == pygame.QUIT: 
            k = False 
        elif event.type == pygame.KEYDOWN: 
            if event.key == pygame.K_ESCAPE: k = False
    screen.blit(crop, (0, 0))#, (100, 100, 100, 30))
    #dt = clock.tick()/1000
    pygame.display.update()   
    screen.fill(0)
pygame.quit()

In [ ]:
pygame.quit()
sys.exit()

In [ ]:
n = 6
(11**n + 1)/2**3

In [ ]:
a=[(4, 3), (9, 8)]
i = len(a)-1
x = a[(i+2)%(len(a))][0]
print(x)

In [ ]:
print(1.123123231-7.9034343)


In [ ]:
(3 + 2*math.sqrt(2))/2

In [ ]:
print(math.degrees(math.atan(math.sqrt(3))))

In [ ]:
print((-4)%5)

In [ ]:
import kivy

In [ ]:
pip install kivy

# 